In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
from collections import deque
import cv2
import sys
import os
import gc
import matplotlib
matplotlib.use("Agg")

In [ ]:
VIDEO_PATH = "path/to/video.mp4"
VIDEO_OUTPUT ="path/to/output_video.avi"
model_location = 'path/to/model.h5'

In [ ]:
def get_model():
    # load the VGG16 network, ensuring the head FC layer sets are left 
    baseModel = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

    headModel = baseModel.output
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(512, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(1, activation="sigmoid")(headModel)

    model = Model(inputs=baseModel.input, outputs=headModel)

    # loop over all layers in the base model and freeze them so they will
    # *not* be updated during the first training process
    for layer in baseModel.layers:
        layer.trainable = False

    # compile our model (this needs to be done after our setting our
    # layers to being non-trainable
    print("[INFO] Model yükleniyor...")
    opt = SGD(lr=1e-6, momentum=0.9)
    model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
    return model

In [ ]:
model = get_model()
model.load_weights(model_location)

In [ ]:
Q = deque(maxlen=128)
print("[INFO] Video İşleniyor...")
vs = cv2.VideoCapture(VIDEO_PATH)
writer = None
(W, H) = (None, None)
 
counter = 0
while True:
    counter+=1

    (grabbed, frame) = vs.read()

    if not grabbed:
        break

    if W is None or H is None:
        (H, W) = frame.shape[:2]

    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224))
    frame = frame.astype("float32")

    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Q.append(preds)

    results = np.array(Q).mean(axis=0)[0] 

    text = f"Hasar Ihtimali %{np.round(results*100,2)}"
    cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,1.25, (0, 255, 0), 5)

    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(VIDEO_OUTPUT, fourcc, 30,(W, H), True)

    writer.write(output)
 
print("[INFO] Video Oluşturuldu...")
writer.release()
vs.release()